# Movie Data Business Problem Part 2
* Steven Phillips
* June 21, 2023
* Part 2: Use an API to extract box office revenue and profit data to add to your IMDB data and perform exploratory data analysis.

## API Calls - Extract budget, revenue, and MPAA Rating (G/PG/PG-13/R), which is also called "Certification".

### Load imports, data, tmdb api key

In [1]:
# imports

import pandas as pd
import os, time,json
import tmdbsimple as tmdb 
from tqdm.notebook import tqdm_notebook


In [2]:
# intialize FOLDER variable 

FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['title_akas.csv.gz', 'title_basics.csv.gz', 'title_ratings.csv.gz']

In [3]:
# load TMDB API key

with open('/Users/steve/Documents/GitHub/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
## Display the keys of the loaded dict
login.keys()

dict_keys(['api-key'])

In [4]:
# initialize TMDB API key

tmdb.API_KEY =  login['api-key']

### create functions to call tmdb info, append new data to a json file

In [5]:
# API call from TMDB function for TMDB movie information

def get_movie_with_rating(movie_id):
    """Adapted from source = https://github.com/celiao/tmdbsimple"""
    # Get the movie object for the current id
    movie = tmdb.Movies(movie_id)
    
    # save the .info .releases dictionaries
    info = movie.info()
    
    releases = movie.releases()
    # Loop through countries in releases
    for c in releases['countries']:
        # if the country abbreviation++US
        if c['iso_3166_1' ] =='US':
            ## save a "certification" key in info with the certification
            info['certification'] = c['certification']
            
    return info

In [6]:
# function to append new data to json file

def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

### test tmdb api movie function

In [7]:
## test get_movie_with_rating function by looping through a list of movie ids
# tt0848228 ("The Avengers") and tt0332280 ("The Notebook")

test_ids = ["tt0848228", "tt0332280"]
results = []
errors = []
for movie_id in test_ids:
    
    try:
        
        movie_info = get_movie_with_rating(movie_id)
        results.append(movie_info)
    
    except Exception as e:
        errors.append([movie_id, e])
    
df_results = pd.DataFrame(results)
df_results.head()

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,"{'id': 86311, 'name': 'The Avengers Collection...",220000000,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",https://www.marvel.com/movies/the-avengers,24428,tt0848228,en,The Avengers,...,1518815515,143,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Some assembly required.,The Avengers,False,7.708,28737,PG-13
1,False,/qom1SZSENdmHFNZBXbtJAU0WTlC.jpg,None,29000000,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...",http://www.newline.com/properties/notebookthe....,11036,tt0332280,en,The Notebook,...,115603229,123,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Behind every great love is a great story.,The Notebook,False,7.880,10381,PG-13


### load in basics data from imdb and extract tmdb info for 2000 and 2001

In [8]:
# Load in the dataframe from project part 1 as basics:

basics = pd.read_csv('/Users/steve/Documents/GitHub/Movie-Data-Analysis-Business-Problem-/Data/title_basics.csv.gz')


In [9]:
# create a list of years to extract

YEARS_TO_GET = [2000,2001]

In [10]:
# initialize the errors list

errors = [ ]

In [14]:
# Start of OUTER loop

for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
    
    #Defining the JSON file to store results for year
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    
    # Check if file exists
    file_exists = os.path.isfile(JSON_FILE)
    
    # If it does not exist: create it
    if file_exists == False:
        
        # save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)
            
    #Saving new year as the current df
    df = basics.loc[ basics['startYear']==YEAR].copy()
    
    # saving movie ids to list
    movie_ids = df['tconst'].copy()
     
    # Load existing data from json into a dataframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)
    
    # filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
    
    #Get index and movie id from list
    # start of INNER Loop retrieving movie info from TMDB using function
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        try:
            # Retrieve then data for the movie id
            temp = get_movie_with_rating(movie_id)  
            # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
            
        except Exception as e:
            errors.append([movie_id, e])
            
    ## after inner loop but within outer loop

    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)
    

print(f"- Total errors: {len(errors)}") 

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/2741 [00:00<?, ?it/s]

Movies from 2001:   0%|          | 0/2874 [00:00<?, ?it/s]

- Total errors: 1083


### explore tmdb info from 2000 and 2001

In [15]:
# Open saved TMDB 2000 file and preview as data frame

df_tmdb_2000 = pd.read_csv("Data/final_tmdb_data_2000.csv.gz", low_memory = False)
df_tmdb_2000.head()

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0113026,0.0,/vMFs7nw6P0bIV1jDsQpxAieAVnH.jpg,NaN,10000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10402, '...",NaN,62127.0,en,The Fantasticks,...,0.0,86.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Try to remember the first time magic happened,The Fantasticks,0.0,5.5,22.0,NaN
2,tt0113092,0.0,NaN,NaN,0.0,"[{'id': 878, 'name': 'Science Fiction'}]",NaN,110977.0,en,For the Cause,...,0.0,100.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,The ultimate showdown on a forbidden planet.,For the Cause,0.0,5.1,8.0,NaN
3,tt0116391,0.0,NaN,NaN,0.0,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",NaN,442869.0,hi,Gang,...,0.0,152.0,"[{'english_name': 'Hindi', 'iso_639_1': 'hi', ...",Released,NaN,Gang,0.0,4.0,1.0,NaN
4,tt0116748,0.0,/wr0hTHwkYIRC82MwNbhOvqrw27N.jpg,NaN,0.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,579396.0,hi,Karobaar,...,0.0,180.0,"[{'english_name': 'Hindi', 'iso_639_1': 'hi', ...",Released,The Business of Love,Karobaar,0.0,7.0,3.0,NaN


In [16]:
# explore tmdb 2000 data frame info

df_tmdb_2000.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2224 entries, 0 to 2223
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   imdb_id                2224 non-null   object 
 1   adult                  2223 non-null   float64
 2   backdrop_path          1039 non-null   object 
 3   belongs_to_collection  137 non-null    object 
 4   budget                 2223 non-null   float64
 5   genres                 2223 non-null   object 
 6   homepage               88 non-null     object 
 7   id                     2223 non-null   float64
 8   original_language      2223 non-null   object 
 9   original_title         2223 non-null   object 
 10  overview               2036 non-null   object 
 11  popularity             2223 non-null   float64
 12  poster_path            1918 non-null   object 
 13  production_companies   2223 non-null   object 
 14  production_countries   2223 non-null   object 
 15  rele

In [17]:
# Open saved TMDB 2001 file and preview as data frame

df_tmdb_2001 = pd.read_csv("Data/final_tmdb_data_2001.csv.gz", low_memory = False)
df_tmdb_2001.head()

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0035423,0.0,/hfeiSfWYujh6MKhtGTXyK3DD4nN.jpg,NaN,48000000.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 14, ...",NaN,11232.0,en,Kate & Leopold,...,76019048.0,118.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,"If they lived in the same century, they'd be p...",Kate & Leopold,0.0,6.331,1196.0,PG-13
2,tt0114447,0.0,NaN,NaN,0.0,"[{'id': 53, 'name': 'Thriller'}, {'id': 28, 'n...",NaN,151007.0,en,The Silent Force,...,0.0,90.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,They left him for dead... They should have fin...,The Silent Force,0.0,5.000,3.0,NaN
3,tt0114722,0.0,NaN,NaN,0.0,"[{'id': 18, 'name': 'Drama'}, {'id': 53, 'name...",NaN,276251.0,es,3 Noches,...,0.0,105.0,"[{'english_name': 'Spanish', 'iso_639_1': 'es'...",Released,NaN,3 Nights,0.0,0.000,0.0,NaN
4,tt0116916,0.0,/rFpHBidSlhjflmnLu7BZilyKeQR.jpg,NaN,0.0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,73549.0,en,The Dark Mist,...,0.0,101.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,NaN,The Dark Mist,0.0,3.500,2.0,PG


In [18]:
# explore tmdb 2001 data frame info

df_tmdb_2001.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2310 entries, 0 to 2309
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   imdb_id                2310 non-null   object 
 1   adult                  2309 non-null   float64
 2   backdrop_path          1058 non-null   object 
 3   belongs_to_collection  122 non-null    object 
 4   budget                 2309 non-null   float64
 5   genres                 2309 non-null   object 
 6   homepage               140 non-null    object 
 7   id                     2309 non-null   float64
 8   original_language      2309 non-null   object 
 9   original_title         2309 non-null   object 
 10  overview               2154 non-null   object 
 11  popularity             2309 non-null   float64
 12  poster_path            1994 non-null   object 
 13  production_companies   2309 non-null   object 
 14  production_countries   2309 non-null   object 
 15  rele